# ML в Биологии
## 1. Анализ данных

In [ ]:
!pip install torchviz

In [ ]:
import torchviz

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.autograd import grad
from torch import nn
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support as all_metrics
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.autonotebook import trange, tqdm

from collections import defaultdict

from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split

from IPython.display import clear_output

In [ ]:
sns.set(palette='Set2', font_scale=1.2)
%matplotlib inline

### Задача 1. Дифференцирование

Для функции

$$ f(x, y, z)=\left(\frac{y^4}{1+e^{-x}}\right)^3 + z$$

С помощью Pytorch:

- постройте вычислительный граф с возможностью считать производные по x, y.

- выведите все узлы полученного графа и их атрибуты (рассмотренные на лекции)

- объясните, почему атрибуты принимают такие значения

Для точки (1, 3, 2):

- посчитайте для функции все возможные первые частные производные методом backward()

- посчитайте для функции все возможные вторые частные производные

Импорт библиотек:

In [ ]:
import torch
from torchviz import make_dot

Построим вычислительный граф для заданной функции:

In [ ]:
x = torch.tensor(1, requires_grad=True, dtype=torch.float)
y = torch.tensor(3, requires_grad=True, dtype=torch.float)
z = torch.tensor(2, requires_grad=True, dtype=torch.float)

In [ ]:
f = (y**4 / (1 + torch.exp(-x)))**3 + z

In [ ]:
make_dot(f, params={'x': x, 'y': y, 'z' : z, 'f': f})

Из данного графа мы сможем увидеть порядок вычисления производной функции f по правилу взятия производной от сложной функции.

Выведем все узлы полученного графа и их атрибуты:

In [ ]:
print('Атрибуты для функции f:\n')
print(f'Значение функции в точке (1, 3, 2): {f.item()}')
print(f'Функция вычисления градиента (последняя): {f.grad_fn}')
print(f'Следит ли PyTorch за градиентами этого тензора: {f.requires_grad}')
print(f'Является ли листом графа? Если нет - вершина: {f.is_leaf}')

print('\nАтрибуты для x:\n')
print(f'Значение x: {x.item()}')
print(f'Градиент для x (пока None): {x.grad}')
print(f'Функция вычисления градиента для x: {x.grad_fn}')
print(f'Является ли x листом вычислительного графа: {x.is_leaf}')

Вычисление первых производных:

In [ ]:
f.backward()

Вывод градиентов (первых производных):

In [ ]:
print(f'Градиенты: \n ∂f/∂x = {x.grad.item()}, ∂f/∂y = {y.grad.item()}, ∂f/∂z = {z.grad.item()}')

Сброс градиентов для дальнейших вычислений:

In [ ]:
x.grad.zero_()
y.grad.zero_()
z.grad.zero_()

Явные выражения для первых производных:

In [ ]:
fx = 3 * y ** 12 * torch.exp(-x) / ((1 + torch.exp(-x)) ** 4)
fy = 12 * y ** 11 / ((1 + torch.exp(-x)) ** 3)

Вычисление вторых производных:

In [ ]:
fx.backward()
print(f'Вторые производные: \n ∂²f/∂x² = {x.grad.item()}, ∂²f/∂x∂y = {y.grad.item()}')

x.grad.zero_()
y.grad.zero_()

fy.backward()
print(f'∂²f/∂y∂x = {x.grad.item()}, ∂²f/∂y² = {y.grad.item()}')

### Задача 2.

**Далее идёт предобработка биологического датасета, которую можно удалить, если вы работаете с физическим. После этого блока (см. ниже) идёт общая часть с созданием, обучением и тестированием нейросети.**

#### Профиль биология

Скачайте [датасет](https://disk.yandex.ru/d/FVoQGn5q1td7Vw), описывающий влияние курения и алкоголя на человека. Создайте и обучите нейросеть, разделяющую эти два класса.

[Описание датасета](https://www.kaggle.com/datasets/sooyoungher/smoking-drinking-dataset)

In [ ]:
!unzip /content/smoking_driking_dataset.zip

In [ ]:
df = pd.read_csv('smoking_driking_dataset_Ver01.csv')
df.head()

In [ ]:
df.info()

Видно, что объекты класса `'sex'` принадлежат типу `'object'`. Заменим `Male` на 1, `Female` на 0:

In [ ]:
df['sex'][df['sex'] == 'Male'] = 1
df['sex'][df['sex'] == 'Female'] = 0
df.head()

Проанализируйте датасет, разделяя его по:
* курит &mdash; 3
* курил, но бросил &mdash; 2
* никогда не курил &mdash; 1

Проведем поверхностный анализ, построив гистограмму:

In [ ]:
smokers = df[df.SMK_stat_type_cd == 3].shape[0]
past_smokers = df[df.SMK_stat_type_cd == 2].shape[0]
no_smokers = df[df.SMK_stat_type_cd == 1].shape[0]

labels = ['smokers', 'past_smokers', 'no_smokers']
values = [smokers, past_smokers, no_smokers]

In [ ]:
sns.barplot(x = labels, y = values, palette = ['blue', 'green', 'orange'])
plt.title('Smoking Status Distribution')

**Вывод:**

Так как нам предстоит проклассифицировать людей с плохими привычками и без них, то нам нужно будет решить задачу бинарной классификации. Исходя из поверхностного анализа, предлагаю объединить курильщиков и бывших курильщиков в одну группу. Тогда наши классы будут вполне всбалансированы.

Попробуем классифицировать людей с плохими привычками и без них. Для этого создайте дополнительный столбец в таблице, который будет содержать информацию о том, имеет ли человек вредные привычки или нет.

**Невероятно, но факт:** употребление алкоголя/пьянство - это плохая привычка!

Поэтому предлагаю создать столбец `Bad_habits` следующим образом:

*   Пьющие, курящие и бросившие курить люди - `True`
*   Непьющие и некурящие - `False`

In [ ]:
df['SMK_stat_type_cd'][df['SMK_stat_type_cd'] == 1] = False
df['SMK_stat_type_cd'][df['SMK_stat_type_cd'] == 2] = True
df['SMK_stat_type_cd'][df['SMK_stat_type_cd'] == 3] = True

df['DRK_YN'][df['DRK_YN'] == 'Y'] = True
df['DRK_YN'][df['DRK_YN'] == 'N'] = False

df['Bad_habits'] = df['SMK_stat_type_cd'] | df['DRK_YN']

df.head()

Теперь смотрим на разделение по привычкам

In [ ]:
bad = df[df.Bad_habits == 1].shape[0]
good = df[df.Bad_habits == 0].shape[0]

labels = ['Bad_habits', 'Good_habits']

sns.barplot(x = labels, y = [bad, good], palette = ['green', 'orange'])
plt.title('Habitis Status Distribution')

**Вывод:**

Сильного дисбаланса не наблюдается. Жить можно.

Какие признаки вы бы использовали для разделения людей по классам? Выберите эти столбцы и создайте наборы train и test с помощью функции train_test_split, а также выделите набор данных для валидации при обучении.

Так как фичей у нас не очень много, то можно посмотреть на матрицу корреляций между признакми, где особое внимание обратим на корреляцию столбца `True` с остальными признаками:

In [ ]:
corr = df[df.columns].corr()

plt.figure(figsize = (15, 12))

sns.heatmap(corr, xticklabels = corr.columns.values, yticklabels = corr.columns.values)

Выведем списки корреляций признаков со столбцом `Bad_habits`:

In [ ]:
cor = df.corr()['Bad_habits']

# Сильная корреляция (0.7 <= r)
strong = cor[abs(cor) >= 0.7].index.tolist()

# Средняя корреляция (0.5 <= r < 0.7)
medium = cor[(abs(cor) >= 0.5) & (abs(cor) < 0.7)].index.tolist()

# Умеренная корреляция (0.3 <= r < 0.5)
moderate = cor[(abs(cor) >= 0.3) & (abs(cor) < 0.5)].index.tolist()

# Слабая корреляция (0.2 <= r < 0.3)
weak = cor[(abs(cor) >= 0.2) & (abs(cor) < 0.3)].index.tolist()

# Очень слабая корреляция (r < 0.2)
so_weak = cor[abs(cor) < 0.2].index.tolist()

print('Сильная корреляция:', strong)
print('Средняя корреляция:', medium)
print('Умеренная корреляция:', moderate)
print('Слабая корреляция:', weak)
print('Очень слабая корреляция:', so_weak)

В качестве фичей возьмем фичи, обладающие слабой и средней корреляцией. `SMK_stat_type_cd` не будем учитывать так как ранее мы использовали это для анализа людей с плохими привычками. Посмотрим на их KDE:

In [ ]:
plt.figure(figsize = (20, 5))

features = ['sex', 'height', 'weight', 'hemoglobin', 'age']
for i, feature in enumerate(features):
    plt.subplot(1, len(features), i + 1)
    if feature == 'sex':
        sns.histplot(data = df, x = feature, hue = 'Bad_habits', multiple = 'stack', stat = 'density', alpha = 0.5)
        plt.title(f'Гистограмма для {feature} по Bad_habits')
    else:
        sns.kdeplot(data = df, x = feature, hue = 'Bad_habits', fill = True)
        plt.title(f'KDE для {feature} по Bad_habits')

plt.tight_layout()
plt.show()

Попробоум провести дальнейший анализ на этих фичах.

In [ ]:
data_features = df[features]
target = df['Bad_habits'].astype(int)

X_train, X_test_valid, y_train, y_test_valid = train_test_split(
    data_features, target, test_size = 0.2, random_state = 42
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test_valid, y_test_valid, test_size = 0.5, random_state = 42)

#### Общая часть

Предлагаю в качесвте уелевой метрики использовать ROC AUC как классическую для задачи бинарной классификации. А в качесвте функции потерь - Binary Cross Entropy.

In [ ]:
class_lim_proba = 0.5 # критерий принадлежности к тому или иному классу

Стандартизируйте данные

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)
X_valid_s = scaler.transform(X_valid)

Далее сформируйте датасет в pytorch-обертке

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 128

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

In [ ]:
X_train_t = torch.from_numpy(X_train_s).to(torch.float32).to(device)
X_test_t  = torch.from_numpy(X_test_s ).to(torch.float32).to(device)
X_val_t   = torch.from_numpy(X_valid_s).to(torch.float32).to(device)

y_train_t = torch.from_numpy(np.array(y_train)).to(torch.float32).to(device)
y_test_t  = torch.from_numpy(np.array(y_test )).to(torch.float32).to(device)
y_val_t   = torch.from_numpy(np.array(y_valid)).to(torch.float32).to(device)

train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True)

loaders = {"train": train_dataloader, 'valid' : val_dataloader}

Напишите вспомогательные функции, в одной из которых будет проходить обучение, а во вторй - валидация. Также в отдельной функцией можно описать отрисовку кривых обучения. На одном графике расположите значение функции потерь на трейне и валидации, а на другом -- значение метрики качества на ваш выбор, также для трейна и валидации)

In [ ]:
def plot_learning_curves(history):
    '''
    Функция для отображения лосса и метрики во время обучения.
    '''
    fig = plt.figure(figsize = (20,7))

    plt.subplot(1,2,1)

    plt.title('Loss', fontsize = 15)

    plt.plot(history['Loss']['train'], label = 'train')
    plt.plot(history['Loss']['valid'], label = 'val')

    plt.xlabel('Epoch', fontsize = 15)

    plt.legend()

    plt.subplot(1,2,2)

    plt.title('ROC AUC', fontsize = 15)

    plt.plot(history['auc']['train'], label = 'train')
    plt.plot(history['auc']['valid'], label = 'val')

    plt.xlabel('Epoch', fontsize = 15)

    plt.legend()
    plt.show()


def train_epoch(model, criterion, opt, loader):
    '''
    Проход по данным для обучения в одной эпохе
    '''
    hist = []
    metr = []
    model.train()

    for x_batch, y_batch in loader:
        opt.zero_grad()
        outp = model(x_batch)
        y_prob = torch.sigmoid(outp)
        y_pred = torch.round(y_prob)

        loss = criterion(y_prob, y_batch.view(-1, 1))
        loss.backward()
        opt.step()

        hist.append(loss.cpu().detach())
        metr.append(roc_auc_score(y_batch.cpu().detach().numpy(), y_prob.cpu().detach().numpy()))

    return np.mean(hist), np.mean(metr)


def test_epoch(model, criterion, opt, loader):
    '''
    Валидация на одной эпохе
    '''
    model.eval()
    hist = []
    metr = []

    for x_batch, y_batch in loader:
        with torch.no_grad():
            outp = model(x_batch)

        y_prob = torch.sigmoid(outp)
        y_pred = torch.round(y_prob)

        loss = criterion(y_prob, y_batch.view(-1, 1))

        hist.append(loss.cpu().detach())
        metr.append(roc_auc_score(y_batch.cpu().detach().numpy(), y_prob.cpu().detach().numpy()))

    return np.mean(hist), np.mean(metr)

def training_loop(model, criterion, opt, loaders, max_epochs):
    '''
    Весь цикл обучения
    '''
    auroc = {"train": [], "valid" : []}
    loss_history = {"train": [], "valid" : []}

    for epoch in trange(max_epochs):
        for k, dataloader in loaders.items():
            if k == 'train':
                his, met = train_epoch(model, criterion, opt, dataloader)
            elif k == 'valid':
                his, met = test_epoch(model, criterion, opt, dataloader)

            auroc[k].append(met)
            loss_history[k].append(his)

        val = 'valid'
        print(f'Epoch: {epoch + 1}. Valid Loss: {loss_history[val][-1].round(5)}')

    metrics = {
        'auc' : auroc,
        'Loss' : loss_history
    }

    return metrics

Создание модели

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(5, 128)
        self.linear2 = nn.Linear(128, 32)
        self.linear3 = nn.Linear(32, 1)
        self.drop = nn.Dropout(0.5)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.linear1(x))
        x = self.drop(x)
        x = self.activation(self.linear2(x))
        x = self.drop(x)
        x = self.linear3(x)
        return x

Обучение

In [ ]:
model = SimpleModel()
model.to(device)

criterion = nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr = 1e-3)

max_epochs = 30

metrics = training_loop(model, criterion, opt, loaders, max_epochs)

In [ ]:
plot_learning_curves(metrics)

Тестирование

In [ ]:
model.eval()

loss, roc_auc = test_epoch(model, criterion, opt, test_dataloader)

print(f'Test loss: {loss}, test ROC_AUC: {roc_auc}')

Попробуем для сравнения обучить моедль  на всех фичах (вдруг scor улучшиться):

In [ ]:
features = df.columns[:-3]
data_features = df[features]
target = df['Bad_habits'].astype(int)

X_train, X_test_valid, y_train, y_test_valid = train_test_split(
    data_features, target, test_size = 0.2, random_state = 42
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test_valid, y_test_valid, test_size = 0.5, random_state = 42)

In [ ]:
scaler = StandardScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)
X_valid_s = scaler.transform(X_valid)

In [ ]:
batch_size = 128
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

X_train_t = torch.from_numpy(X_train_s).to(torch.float32).to(device)
X_test_t  = torch.from_numpy(X_test_s ).to(torch.float32).to(device)
X_val_t   = torch.from_numpy(X_valid_s).to(torch.float32).to(device)
y_train_t = torch.from_numpy(np.array(y_train)).to(torch.float32).to(device)
y_test_t  = torch.from_numpy(np.array(y_test )).to(torch.float32).to(device)
y_val_t   = torch.from_numpy(np.array(y_valid)).to(torch.float32).to(device)


train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True)


loaders = {"train": train_dataloader, 'valid' : val_dataloader}

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(22, 128)
        self.linear2 = nn.Linear(128, 32)
        self.linear3 = nn.Linear(32, 1)
        self.drop = nn.Dropout(0.5)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.linear1(x))
        x = self.drop(x)
        x = self.activation(self.linear2(x))
        x = self.drop(x)
        x = self.linear3(x)
        return x

In [ ]:
model = SimpleModel()
model.to(device)

criterion = nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr = 1e-3)

max_epochs = 30

metrics = training_loop(model, criterion, opt, loaders, max_epochs)

In [ ]:
plot_learning_curves(metrics)

In [ ]:
model.eval()

loss, auc = test_epoch(model, criterion, opt, test_dataloader)

print(f'Test loss: {loss}, test ROC AUC: {auc}')

**Выводы:**

Мы обработали данные о людях с вредными и здоровыми привычками и успешно обучили полносвязную модель для классификации. Модель показывает достойные результаты (ROC AUC > 0.5).

Графики лосса свидетельствуют, что переобучения не произошло. Возможно, улучшение гиперпараметров, таких как размер скрытых слоёв, скорость обучения, использование дропаута, количество эпох или выбор функции активации, могло бы повысить точность и улучшить снижение лосса. Тем не менее, полученный результат уже является удовлетворительным и достаточным для текущих целей.